# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [18]:
with open("oocl_sales.txt", encoding='utf-8') as f:
    oocl_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [19]:
from langchain.text_splitter import CharacterTextSplitter

In [20]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [21]:
docs = text_splitter.create_documents([oocl_sales])

In [22]:
docs[0]

Document(page_content='[客户问题] 我可以在网上追踪我的货物吗？ \n[客服回答] 当然，我们提供在线追踪服务，您可以通过提单号随时查看货物状态。')

In [23]:
len(docs)

110

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [30]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [31]:
query = "危险物品能够运输吗"

In [32]:
answer_list = db.similarity_search(query)

In [33]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 你们能处理危险品运输吗？ 
[客服回答] 我们有专业的团队和设备处理危险品运输，确保符合所有安全规定。

[客户问题] 你们的船只是否支持危险品运输？
[客服回答] 是的，我们的部分船只专门用于危险品运输，并遵守所有相关的安全规定。

[客户问题] 我的货物在运输过程中是否可以进行转运？
[客服回答] 是的，我们提供转运服务，确保您的货物能够通过最佳路线到达目的地。

[客户问题] 你们提供货物打托服务吗？
[客服回答] 是的，我们提供货物打托服务，确保货物在装卸过程中的稳定性和安全性。



In [34]:
db.save_local("oocl_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [35]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [36]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002875FD80D60>, search_kwargs={'k': 3})

In [37]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\software\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 你们能处理危险品运输吗？ 
[客服回答] 我们有专业的团队和设备处理危险品运输，确保符合所有安全规定。

[客户问题] 你们的船只是否支持危险品运输？
[客服回答] 是的，我们的部分船只专门用于危险品运输，并遵守所有相关的安全规定。

[客户问题] 我的货物在运输过程中是否可以进行转运？
[客服回答] 是的，我们提供转运服务，确保您的货物能够通过最佳路线到达目的地。



In [38]:
docs = topK_retriever.get_relevant_documents("什么时候能到达目的地？")

In [39]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 你们如何保证货物按时到达？
[客服回答] 我们通过严格的航线管理和时刻监控，确保货物能够尽可能按时到达。

[客户问题] 我可以要求货物到达前通知我吗？ 
[客服回答] 当然，我们可以在货物到达前通知您，以便您做好接收准备。

[客户问题] 我可以指定货物交付的时间吗？
[客服回答] 我们会尽力满足您的交付时间要求，请在订舱时提供具体信息。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [41]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [42]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 你们能处理危险品运输吗？ 
[客服回答] 我们有专业的团队和设备处理危险品运输，确保符合所有安全规定。

[客户问题] 你们的船只是否支持危险品运输？
[客服回答] 是的，我们的部分船只专门用于危险品运输，并遵守所有相关的安全规定。

[客户问题] 我的货物在运输过程中是否可以进行转运？
[客服回答] 是的，我们提供转运服务，确保您的货物能够通过最佳路线到达目的地。

[客户问题] 你们提供货物打托服务吗？
[客服回答] 是的，我们提供货物打托服务，确保货物在装卸过程中的稳定性和安全性。



### 提取向量数据库中的`销售回答`

In [43]:
docs = retriever.get_relevant_documents(query)

In [44]:
docs[0].page_content

'[客户问题] 你们能处理危险品运输吗？ \n[客服回答] 我们有专业的团队和设备处理危险品运输，确保符合所有安全规定。'

In [50]:
docs[0].page_content.split("[客服回答] ")

['[客户问题] 你们能处理危险品运输吗？ \n', '我们有专业的团队和设备处理危险品运输，确保符合所有安全规定。']

In [51]:
ans = docs[0].page_content.split("[客服回答] ")[-1]

In [52]:
ans

'我们有专业的团队和设备处理危险品运输，确保符合所有安全规定。'

#### 尝试各种问题

In [53]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[客服回答] ")[-1] for doc in docs]

    return ans_list

In [54]:
query = "是否有提供清关服务"

print(sales(query))

[]


D:\software\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [55]:
print(sales(query, 0.75))

['是的，我们提供海关清关服务，帮助您的货物顺利通过海关。', '是的，我们提供全程的门到门运输解决方案，为您提供便捷。']


In [56]:
query = "运费贵吗"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\software\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []

score:0.75 ans: ['运费根据货物的重量、体积和目的地来计算，我可以为您详细解释。', '您可以通过我们的在线报价工具或直接联系客服来获取详细的运输报价。', '是的，我们提供上门取货服务，为您节省时间和精力。', '是的，我们提供门到门的运输服务，从发货地到目的地的全程物流解决方案。']

score:0.5 ans: ['运费根据货物的重量、体积和目的地来计算，我可以为您详细解释。', '您可以通过我们的在线报价工具或直接联系客服来获取详细的运输报价。', '是的，我们提供上门取货服务，为您节省时间和精力。', '是的，我们提供门到门的运输服务，从发货地到目的地的全程物流解决方案。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [57]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [59]:
qa_chain({"query": "你们和其他船运公司相比有什么优势？"})

D:\software\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们和其他船运公司相比有什么优势？',
 'result': '很抱歉，但我不属于任何船运公司，也没有特定的业务信息。我是一个人工智能助手，用于提供信息和解答问题。如果您有关于船运行业的一般问题，或者需要了解船运服务的某些方面，我会尽力提供帮助。如果您正在寻找具体公司的服务优势，我建议直接访问该公司的官方网站或联系他们的客户服务部门获取最准确的信息。'}

In [32]:
qa_chain({"query": "小区吵不吵"})

{'query': '小区吵不吵',
 'result': '这个小区特别注重居住体验，有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。所以，小区应该不会太吵。'}

In [33]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


## 加载 FAISS 向量数据库已有结果

In [62]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("oocl_sale", OpenAIEmbeddings(), allow_dangerous_deserialization = True)

In [63]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

D:\software\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [70]:
qa_chain({"query": "我想买一条船"})

D:\software\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买一条船',
 'result': '这不是一个问题，你可以提供更多的信息吗？例如，你正在寻找什么类型的船，预算是多少，或者你在哪里购买。',
 'source_documents': []}

In [65]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [66]:
qa_chain({"query": "我想买别墅，你们有么"})

D:\software\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮助您购买别墅，因为我是一个人工智能助手，我没有实体商品或房产。我可以帮助回答有关别墅的问题，或者提供有关购买别墅的一些普遍建议。'}

In [67]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [68]:
result = qa_chain({"query": "我想买别墅，你们有么"})

D:\software\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [69]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我无法帮助你购买别墅。我是一个人工智能助手，我可以提供信息和回答问题，但我不能进行实际的购买或销售活动。',
 'source_documents': []}